## **竞赛地址：** [“达观杯”文本智能处理挑战赛](http://www.dcjingsai.com/common/cmpt/%E2%80%9C%E8%BE%BE%E8%A7%82%E6%9D%AF%E2%80%9D%E6%96%87%E6%9C%AC%E6%99%BA%E8%83%BD%E5%A4%84%E7%90%86%E6%8C%91%E6%88%98%E8%B5%9B_%E7%AB%9E%E8%B5%9B%E4%BF%A1%E6%81%AF.html)

## [参考文章](https://zhuanlan.zhihu.com/p/30455047)

求个关注，star

In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix

import os
import pickle

from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn import svm
from sklearn.metrics import f1_score
from sklearn.model_selection import StratifiedKFold,KFold

In [2]:
from sklearn.feature_selection import SelectKBest, chi2

In [3]:
import warnings
warnings.filterwarnings("ignore")

## 读取训练集和测试集数据

由于数据集较大，我们采用 pandas 分块读取。

In [12]:
reader = pd.read_csv('./new_data/train_set.csv',iterator=True)

chunks = []
loop = True
while loop:
    try:
        chunk = reader.get_chunk(20000)[['id','word_seg','class']]
        chunks.append(chunk)
    except StopIteration:
        loop = False
        print("Iteration is stopped")
        
train = pd.concat(chunks,axis=0)
# train = train.drop_duplicates(subset='word_seg')

Iteration is stopped


In [13]:
reader2 = pd.read_csv('./new_data/test_set.csv',iterator=True)

chunks = []
loop = True
while loop:
    try:
        chunk = reader2.get_chunk(20000)[['id','word_seg']]
        chunks.append(chunk)
    except StopIteration:
        loop = False
        print("Iteration is stopped")
        
test = pd.concat(chunks,axis=0)

Iteration is stopped


### 回收内存

In [14]:
import gc
del chunks
gc.collect()

306

### 特征选择1、使用卡方检验挑选重要 一元词组

In [15]:
train = train.sample(frac=1)
train['class'] = train['class'] -1

label = train['class'].values
test_id = test['id'].values

vec = TfidfVectorizer(ngram_range=(1,1),min_df=5, max_df=0.8,use_idf=1,smooth_idf=1, sublinear_tf=1)

trainX = vec.fit_transform(train['word_seg'])

feature_names = vec.get_feature_names()

/Users/wangle/anaconda3/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:1089: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


In [17]:
len(feature_names), trainX.shape

(207512, (102277, 207512))

In [18]:
ch2 = SelectKBest(chi2, k=150000)
trainX = ch2.fit_transform(trainX,label)

feature_names = [feature_names[i] for i in ch2.get_support(indices=True)]

In [19]:
len(feature_names)

150000

### 特征选择2、计算训练集中每个词的逆文档评率 idf，过滤掉一些 出现次数少和 idf 值较小的词，用于做特征选择。

In [20]:
corpus = []
for doc in train['word_seg'].tolist() + test['word_seg'].tolist():
    doc = doc.split()
    corpus.append(doc)

In [22]:
def make_idf_vocab(train_data):
    if os.path.exists('./new_data/vocab.pkl'):
        vocab = pickle.load(open('./new_data/vocab.pkl','rb'))
    else:
        word_to_doc = {}
        idf = {}
        total_doc_num = float(len(train_data))

        for doc in train_data:
            for word in set(doc):
                if word not in word_to_doc.keys():
                    word_to_doc[word] = 1
                else:
                    word_to_doc[word] += 1

        for word in word_to_doc.keys():
            if word_to_doc[word] > 10:
                idf[word] = np.log(total_doc_num/(word_to_doc[word]+1))

        sort_idf = sorted(idf.items(),key=lambda x:x[1])
        vocab = [x[0] for x in sort_idf]
        pickle.dump(vocab,open('./new_data/vocab.pkl','wb'))
    return vocab

In [23]:
vocab = make_idf_vocab(corpus)

In [24]:
len(vocab)

187577

In [9]:
del corpus
gc.collect()

0

### 提取卡方检验选择的特征和 idf 选择的特征的共同特征

In [25]:
vocab = vocab[:130000]

vocab = set(vocab) & set(feature_names)

len(vocab)

108662

In [26]:
vocab_dict = {w:1 for w in vocab}
def filter_word(x):
    x = x.split()
    x = [w for w in x if w in vocab_dict.keys()]
    return ' '.join(x)

train['word_seg'] = train['word_seg'].map(lambda x : filter_word(x))
test['word_seg'] = test['word_seg'].map(lambda x : filter_word(x))

### ----------------------------------

## 特征构造

### 提取 TF-IDF 特征

In [27]:
vec = TfidfVectorizer(ngram_range=(1,2),min_df=5, max_df=0.8,use_idf=1,smooth_idf=1, sublinear_tf=1)

trainX = vec.fit_transform(train['word_seg'])
testX = vec.transform(test['word_seg'])

In [28]:
testX.shape, trainX.shape

((102277, 1509583), (102277, 1509583))

### 提取词频特征

In [29]:
from sklearn.feature_extraction.text import CountVectorizer
from scipy.sparse import csr_matrix, hstack

cv = CountVectorizer(max_features=500000,max_df=0.7, min_df=3, lowercase=False ,ngram_range=(2,3))

trainX_cv = cv.fit_transform(train['word_seg'])
testX_cv = cv.transform(test['word_seg'])

In [30]:
trainX = hstack([trainX_cv,trainX])
testX = hstack([testX_cv,testX])

In [5]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler(with_mean=False)
trainX = scaler.fit_transform(trainX)
testX = scaler.transform(testX)

In [6]:
trainX.shape, testX.shape

((102277, 1200000), (102277, 1200000))

### 采用卡方检验选择特征，降维

In [31]:
ch2 = SelectKBest(chi2, k=1200000)
trainX = ch2.fit_transform(trainX,label)
testX = ch2.transform(testX)

## SVC 模型

### 使用 SVC 模型对训练集进行 10 折交叉验证，最后将10组结果进行投票确定最终预测结果

In [ ]:
test_svc = np.zeros((testX.shape[0],5))

kf = StratifiedKFold(n_splits=10,random_state=233,shuffle=True)
for i,(train_index, val_index) in enumerate(kf.split(trainX,label)):
    if i > 4:
        break
    X_train = trainX[train_index]
    y_train = label[train_index]
    X_val = trainX[val_index]
    y_val = label[val_index]
    
    svc_clf = svm.LinearSVC(class_weight='balanced')
    svc_clf.fit(X_train, y_train)
    pred_val = svc_clf.predict(X_val)
    print(f1_score(y_val, pred_val, average='macro'))
    
    test_svc[:,i] = svc_clf.predict(testX)
    

In [25]:
from collections import Counter

res_bag = []
for r in test_svc:
    res_bag.append(int( Counter(r).most_common(1)[0][0]))

### 提交结果

In [26]:
res = pd.DataFrame({"class":res_bag,'id':test_id})
res['class'] = (res['class']+1).astype(int)
res[["id","class"]].to_csv('svc5cv_baseline.csv',index=None)

## LR 模型

### 使用逻辑斯提回归模型进行10折交叉验证，并且对10次预测结果的概率值进行平均，取最大平均概率为预测结果

In [10]:
test_lr = np.zeros((testX.shape[0],19))
seed = 0
kf = StratifiedKFold(n_splits=10,random_state=233,shuffle=True)
for train_index, val_index in kf.split(trainX,label):
    X_train = trainX[train_index]
    y_train = label[train_index]
    X_val = trainX[val_index]
    y_val = label[val_index]
    
    seed += 10
    clf = LogisticRegression(C=4, dual=True,n_jobs=4,multi_class='ovr',random_state=seed)
    clf.fit(X_train, y_train)
    predi=clf.predict(X_val)
    print(f1_score(y_val, predi, average='macro'))
    
    test_lr += clf.predict_proba(testX)

0.7724444155150814
0.7756671627886585
0.7719812664813792
0.7790966001032718
0.7713963296159765
0.7728899380307235
0.7717692943858951
0.7734096482682434
0.7744500831825549
0.7778699282160826


In [11]:
print(classification_report(y_val, predi,target_names=['%s' % i for i in range(20)]))

             precision    recall  f1-score   support

          0       0.62      0.60      0.61       537
          1       0.83      0.73      0.78       290
          2       0.85      0.90      0.87       831
          3       0.88      0.80      0.84       382
          4       0.85      0.76      0.80       236
          5       0.95      0.90      0.92       688
          6       0.80      0.66      0.72       303
          7       0.74      0.78      0.76       697
          8       0.91      0.95      0.93       767
          9       0.73      0.73      0.73       496
         10       0.72      0.68      0.70       357
         11       0.70      0.66      0.68       532
         12       0.72      0.83      0.77       790
         13       0.77      0.83      0.80       674
         14       0.89      0.91      0.90       751
         15       0.78      0.62      0.69       322
         16       0.77      0.76      0.76       309
         17       0.86      0.91      0.88   

### 提交结果

In [13]:
preds = np.argmax(test_lr,axis=1)
preds = preds + 1

res = pd.DataFrame({"class":preds,'id':test_id})
res['class'] = res['class'].astype(int)
res[["id","class"]].to_csv('lr10cv_baseline2.csv',index=None)

### 保存训练集文件

In [5]:
from scipy import sparse

pickle.dump(test_id,open('./new_data/test_id.pkl','wb'))
pickle.dump(label,open('./new_data/label.pkl','wb'))

sparse.save_npz("./new_data/trainX.npz", trainX)
sparse.save_npz("./new_data/testX.npz", testX)

## LGBM 模型

效果好像不好，训练速度慢，准确率低，训练完之后就 die 了，没提交结果。

In [6]:
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectFromModel
import pickle

In [4]:
from scipy import sparse

test_id = pickle.load(open('./new_data/test_id.pkl','rb'))
label = pickle.load(open('./new_data/label.pkl','rb'))

trainX = sparse.load_npz("./new_data/trainX.npz")
testX = sparse.load_npz("./new_data/testX.npz")

### 使用模型选择特征

In [7]:
model = LogisticRegression(solver='liblinear',penalty='l1')
sfm = SelectFromModel(model, threshold='mean')
trainX = sfm.fit_transform(trainX, label)
testX = sfm.transform(testX)

print(trainX.shape)

(102277, 62732)


In [ ]:
X_train, X_val, y_train, y_val = train_test_split(trainX,label,\
                                    test_size=0.10, random_state=24)

params = {
        'task': 'train','boosting_type': 'gbdt','objective': 'multiclass','num_class':19,
        'metric': {'multi_logloss','multi_error'},'max_depth':-1,'num_leaves':131,
        'min_data_in_leaf':50,'learning_rate': 0.02,'lambda_l2':0.1,
        'feature_fraction': 0.7,'bagging_fraction': 0.7,'bagging_freq': 0,
        'verbosity': 20}

lgb_train = lgb.Dataset(X_train,label=y_train,feature_name='auto')
lgb_eval = lgb.Dataset(X_val,label=y_val,feature_name='auto', reference=lgb_train)
clf = lgb.train(params,lgb_train,num_boost_round=12000,valid_sets=lgb_eval,early_stopping_rounds=50)

pred = clf.predict(testX,num_iteration=clf.best_iteration)

[1]	valid_0's multi_logloss: 2.86157	valid_0's multi_error: 0.37016
Training until validation scores don't improve for 50 rounds.
[2]	valid_0's multi_logloss: 2.78781	valid_0's multi_error: 0.334181
[3]	valid_0's multi_logloss: 2.72252	valid_0's multi_error: 0.322155
[4]	valid_0's multi_logloss: 2.66448	valid_0's multi_error: 0.317951
[5]	valid_0's multi_logloss: 2.60995	valid_0's multi_error: 0.312573
[6]	valid_0's multi_logloss: 2.5601	valid_0's multi_error: 0.312378
[7]	valid_0's multi_logloss: 2.51462	valid_0's multi_error: 0.307098
[8]	valid_0's multi_logloss: 2.47198	valid_0's multi_error: 0.307196
[9]	valid_0's multi_logloss: 2.43104	valid_0's multi_error: 0.306707
[10]	valid_0's multi_logloss: 2.3925	valid_0's multi_error: 0.304556
[11]	valid_0's multi_logloss: 2.35639	valid_0's multi_error: 0.304654
[12]	valid_0's multi_logloss: 2.32183	valid_0's multi_error: 0.301819
[13]	valid_0's multi_logloss: 2.28942	valid_0's multi_error: 0.301036
[14]	valid_0's multi_logloss: 2.25881	va

[118]	valid_0's multi_logloss: 1.20423	valid_0's multi_error: 0.283633
[119]	valid_0's multi_logloss: 1.2008	valid_0's multi_error: 0.284122
[120]	valid_0's multi_logloss: 1.19728	valid_0's multi_error: 0.283438
[121]	valid_0's multi_logloss: 1.19396	valid_0's multi_error: 0.28334
[122]	valid_0's multi_logloss: 1.19054	valid_0's multi_error: 0.283535
[123]	valid_0's multi_logloss: 1.18718	valid_0's multi_error: 0.283144
[124]	valid_0's multi_logloss: 1.18398	valid_0's multi_error: 0.282851
[125]	valid_0's multi_logloss: 1.18078	valid_0's multi_error: 0.282264
[126]	valid_0's multi_logloss: 1.17772	valid_0's multi_error: 0.28246
[127]	valid_0's multi_logloss: 1.17463	valid_0's multi_error: 0.282264
[128]	valid_0's multi_logloss: 1.17149	valid_0's multi_error: 0.282167
[129]	valid_0's multi_logloss: 1.16849	valid_0's multi_error: 0.282949
[130]	valid_0's multi_logloss: 1.16545	valid_0's multi_error: 0.28246
[131]	valid_0's multi_logloss: 1.16246	valid_0's multi_error: 0.282167
[132]	vali

[234]	valid_0's multi_logloss: 0.983051	valid_0's multi_error: 0.271118
[235]	valid_0's multi_logloss: 0.982006	valid_0's multi_error: 0.270923
[236]	valid_0's multi_logloss: 0.981026	valid_0's multi_error: 0.271118
[237]	valid_0's multi_logloss: 0.980049	valid_0's multi_error: 0.271118
[238]	valid_0's multi_logloss: 0.979001	valid_0's multi_error: 0.271118
[239]	valid_0's multi_logloss: 0.978111	valid_0's multi_error: 0.27063
[240]	valid_0's multi_logloss: 0.977132	valid_0's multi_error: 0.270239
[241]	valid_0's multi_logloss: 0.976155	valid_0's multi_error: 0.270239
[242]	valid_0's multi_logloss: 0.975199	valid_0's multi_error: 0.270043
[243]	valid_0's multi_logloss: 0.974271	valid_0's multi_error: 0.26975
[244]	valid_0's multi_logloss: 0.973373	valid_0's multi_error: 0.270043
[245]	valid_0's multi_logloss: 0.972398	valid_0's multi_error: 0.26975
[246]	valid_0's multi_logloss: 0.97151	valid_0's multi_error: 0.269847
[247]	valid_0's multi_logloss: 0.970654	valid_0's multi_error: 0.270

[349]	valid_0's multi_logloss: 0.904704	valid_0's multi_error: 0.262026
[350]	valid_0's multi_logloss: 0.904193	valid_0's multi_error: 0.262124
[351]	valid_0's multi_logloss: 0.903723	valid_0's multi_error: 0.262026
[352]	valid_0's multi_logloss: 0.903241	valid_0's multi_error: 0.261732
[353]	valid_0's multi_logloss: 0.902824	valid_0's multi_error: 0.261341
[354]	valid_0's multi_logloss: 0.90236	valid_0's multi_error: 0.261341
[355]	valid_0's multi_logloss: 0.901996	valid_0's multi_error: 0.261341
[356]	valid_0's multi_logloss: 0.901553	valid_0's multi_error: 0.261146
[357]	valid_0's multi_logloss: 0.901091	valid_0's multi_error: 0.261244
[358]	valid_0's multi_logloss: 0.900624	valid_0's multi_error: 0.260657
[359]	valid_0's multi_logloss: 0.900235	valid_0's multi_error: 0.26095
[360]	valid_0's multi_logloss: 0.899813	valid_0's multi_error: 0.260853
[361]	valid_0's multi_logloss: 0.899473	valid_0's multi_error: 0.26095
[362]	valid_0's multi_logloss: 0.899004	valid_0's multi_error: 0.26

[464]	valid_0's multi_logloss: 0.868909	valid_0's multi_error: 0.255475
[465]	valid_0's multi_logloss: 0.868693	valid_0's multi_error: 0.255182
[466]	valid_0's multi_logloss: 0.868529	valid_0's multi_error: 0.255084
[467]	valid_0's multi_logloss: 0.868328	valid_0's multi_error: 0.255182
[468]	valid_0's multi_logloss: 0.86812	valid_0's multi_error: 0.255573
[469]	valid_0's multi_logloss: 0.867947	valid_0's multi_error: 0.255475
[470]	valid_0's multi_logloss: 0.86776	valid_0's multi_error: 0.255671
[471]	valid_0's multi_logloss: 0.867568	valid_0's multi_error: 0.255475
[472]	valid_0's multi_logloss: 0.867354	valid_0's multi_error: 0.255671
[473]	valid_0's multi_logloss: 0.867183	valid_0's multi_error: 0.255084
[474]	valid_0's multi_logloss: 0.866965	valid_0's multi_error: 0.255182
[475]	valid_0's multi_logloss: 0.866831	valid_0's multi_error: 0.254791
[476]	valid_0's multi_logloss: 0.866651	valid_0's multi_error: 0.254986
[477]	valid_0's multi_logloss: 0.86648	valid_0's multi_error: 0.25

[579]	valid_0's multi_logloss: 0.853705	valid_0's multi_error: 0.250587
[580]	valid_0's multi_logloss: 0.853593	valid_0's multi_error: 0.250489
[581]	valid_0's multi_logloss: 0.853502	valid_0's multi_error: 0.250489
[582]	valid_0's multi_logloss: 0.853458	valid_0's multi_error: 0.250489
[583]	valid_0's multi_logloss: 0.853406	valid_0's multi_error: 0.250489
[584]	valid_0's multi_logloss: 0.85335	valid_0's multi_error: 0.250489
[585]	valid_0's multi_logloss: 0.853309	valid_0's multi_error: 0.250293
[586]	valid_0's multi_logloss: 0.853201	valid_0's multi_error: 0.250293
[587]	valid_0's multi_logloss: 0.853119	valid_0's multi_error: 0.250293
[588]	valid_0's multi_logloss: 0.853076	valid_0's multi_error: 0.25
[589]	valid_0's multi_logloss: 0.853016	valid_0's multi_error: 0.250098
[590]	valid_0's multi_logloss: 0.852919	valid_0's multi_error: 0.25
[591]	valid_0's multi_logloss: 0.852871	valid_0's multi_error: 0.25
[592]	valid_0's multi_logloss: 0.852834	valid_0's multi_error: 0.25
[593]	val

[694]	valid_0's multi_logloss: 0.849872	valid_0's multi_error: 0.247849
[695]	valid_0's multi_logloss: 0.849865	valid_0's multi_error: 0.24824
[696]	valid_0's multi_logloss: 0.849866	valid_0's multi_error: 0.248045
[697]	valid_0's multi_logloss: 0.84988	valid_0's multi_error: 0.247849
[698]	valid_0's multi_logloss: 0.849879	valid_0's multi_error: 0.247751
[699]	valid_0's multi_logloss: 0.849839	valid_0's multi_error: 0.247947
[700]	valid_0's multi_logloss: 0.849802	valid_0's multi_error: 0.247654
[701]	valid_0's multi_logloss: 0.849833	valid_0's multi_error: 0.247751
[702]	valid_0's multi_logloss: 0.849848	valid_0's multi_error: 0.247751
[703]	valid_0's multi_logloss: 0.849901	valid_0's multi_error: 0.247849
[704]	valid_0's multi_logloss: 0.849866	valid_0's multi_error: 0.247849
[705]	valid_0's multi_logloss: 0.849877	valid_0's multi_error: 0.247654
[706]	valid_0's multi_logloss: 0.849894	valid_0's multi_error: 0.247654
[707]	valid_0's multi_logloss: 0.849943	valid_0's multi_error: 0.2